# Create Image with Model 

Download some realife images.

Like this [6gb coco data](http://images.cocodataset.org/zips/val2014.zip)

Or this [19gb coco data](http://images.cocodataset.org/zips/unlabeled2017.zip) will work even better

In [1]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader

### A spacialized dataset for the task

In [2]:
from glob import glob
from PIL import Image
class super_res(Dataset):
    def __init__(self, root_dir):
        """
        A dataset to spit out low and high version of the same picture
        root_dir: directory of the images
        
        """
        self.root_dir = root_dir if root_dir[-1]=="/" else root_dir+"/"
        
        self.resize1 = transforms.Resize((128,128)) # function: resize to smaller
        self.resize2 = transforms.Resize((256,256)) # function: resize to bigger
        
        self.data_aug = transforms.Compose([
            transforms.RandomHorizontalFlip(), # function flip
            transforms.RandomAffine(5), # function rotate
                              ])
        self.toTensor = transforms.ToTensor()
        self.imgs = glob(self.root_dir + "*.jpg")
        
    def __len__(self):
        return len(self.imgs)
        
    def __getitem__(self,idx):
        img_name = self.imgs[idx]
        img = Image.open(img_name,mode = "r").convert('RGB')
        image_l = self.toTensor(self.resize1(self.data_aug(img))) # Low resolution version
        image_h = self.toTensor(self.resize2(self.data_aug(img))) # High resolution version

        return image_l, image_h

In [3]:
from torch import cuda
CUDA = cuda.is_available()
DATA = "/data/unlabeled2017/"

In [4]:
ds = super_res(DATA)
dl = DataLoader(ds,batch_size=1,shuffle = True)
generator = iter(dl)
next(generator)
# len(dl)

[tensor([[[[ 0.0000,  0.0000,  0.0000,  ...,  0.2863,  0.0039,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.3137,  0.0078,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.3020,  0.0078,  0.0000],
           ...,
           [ 0.0000,  0.0039,  0.1255,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0039,  0.1412,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0039,  0.1216,  ...,  0.0000,  0.0000,  0.0000]],
 
          [[ 0.0000,  0.0000,  0.0000,  ...,  0.3176,  0.0039,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.3529,  0.0078,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.3451,  0.0078,  0.0000],
           ...,
           [ 0.0000,  0.0039,  0.1412,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0039,  0.1529,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0039,  0.1294,  ...,  0.0000,  0.0000,  0.0000]],
 
          [[ 0.0000,  0.0000,  0.0000,  ...,  0.3529,  0.0078,  0.0000],
           [ 

### Model Components

In [5]:
from torch import nn
import torch

In [6]:
def conv_layer(in_,out_,ks, stride,activation = True):
    layers = [
        nn.Conv2d(in_,out_,kernel_size = ks, stride = stride, padding = ks//2, bias = False),
        nn.BatchNorm2d(out_),
    ]
    if activation: layers.append(nn.LeakyReLU())
    return nn.Sequential(*layers)

In [7]:
conv_layer(3,4,3,1)

Sequential(
  (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.01)
)

In [8]:
class res_block(nn.Module):
    def __init__(self,nb_filter):
        """
        simple resnet block
        """
        super(res_block,self).__init__()
        self.nb_filter = nb_filter
        self.clayer1 = conv_layer(self.nb_filter,self.nb_filter,3,(1,1))
        self.clayer2 = conv_layer(self.nb_filter,self.nb_filter,3,(1,1),activation = False)
    
    def forward(self,x):
        x2 = self.clayer2(self.clayer1(x))
        return x2+x

In [9]:
def up_block(nb_filters):
    layers = [nn.Upsample(scale_factor=2),
                conv_layer(nb_filters,nb_filters,3,stride = (1,1),activation=True),
             ]
    return nn.Sequential(*layers)

In [10]:
class small2big(nn.Module):
    def __init__(self,nb_filters=64):
        super(small2big,self).__init__()
        self.nb_filters = nb_filters
        self.input_conv = conv_layer(in_=3,out_=self.nb_filters,ks=9,stride=1)
        self.res_blocks = nn.Sequential(*list(res_block(self.nb_filters) for i in range(6)))
        self.up = nn.Sequential(*list(up_block(self.nb_filters) for i in range(1)))
        self.output_conv = nn.Sequential(*[
            nn.Conv2d(in_channels=self.nb_filters,out_channels=3,kernel_size=3,stride=(1,1),padding = 1),
            nn.Sigmoid(),
                                          ])
        
    def forward(self,x):
        x = self.input_conv(x)
        x = self.res_blocks(x)
        x = self.up(x)
        x = self.output_conv(x)
        return x

In [11]:
super_res_model = small2big()

#### Test Scale and Size

In [12]:
super_res_model(torch.rand(2,3,128,128)).size()

torch.Size([2, 3, 256, 256])

In [13]:
if CUDA:
    super_res_model = super_res_model.cuda()
from torch.optim import Adam
loss_func = nn.MSELoss()
opt = Adam(super_res_model.parameters())

In [14]:
from ray.matchbox import Trainer

In [15]:
def save_model(model, path):
    """
    model:pytorch model
    path:save to path, end with pkl
    """
    torch.save(model.state_dict(), path)
    
def load_model(model,path):
    model.load_state_dict(torch.load(path))

In [ ]:
def action(*args,**kwargs):
    low,high = args[0]
    if CUDA:
        low,high = low.cuda(),high.cuda()
    opt.zero_grad()
    high_ = super_res_model(low)
    
    loss = loss_func(high_,high)
    loss.backward()
    opt.step()
    
    if kwargs["ite"]%10==9:
        save_model(super_res_model,"super_res_0.0.1.npy")
    return {"loss":loss.item()}

In [ ]:
trainer = Trainer(ds ,batch_size = 32, print_on= 5)
trainer.action = action

trainer.train(1)

⭐[ep_0_i_69]	loss	0.043:   2%|▏         | 71/3857 [02:13<1:58:13,  1.87s/it]